In [1]:
import os
import numpy as np
import pandas as pd

USE_COL = ['死者姓名','证件号码','死亡时间','根本死亡原因','根本死因ICD编码']

file_path = r'G:/2021.2.7接手（桌面以前文档）/死因工作/数据备份/汇总12-21/'
dfs = []

for file in os.listdir(file_path):
    dir = os.path.join(file_path,file)
    dfs.append(pd.read_csv(dir,encoding='ANSI',usecols=USE_COL))

df = pd.concat(dfs)
df.to_excel(file_path+'total.xlsx',index=False)

## plotly导出pdf(超清图，体积也很小)需要三个库支持


`conda install -c plotly plotly-orca`

`conda install psutil requests`

```python
figs.write_image('image/team.pdf') # 超清图体积大概52KB

figs.write_html('web/www.html') # 体积大概3MB,且支持动态网页,体积依然3MB
```

## plotly_express 0.4 是对plotly一些功能的高级封装

`conda install -c plotly plotly_express==0.4.1`

In [1]:
import plotly_express as px

### 1.甘特图

#### 1.基于px实现
timeline

In [6]:
import pandas as pd
df1 = pd.read_excel('gantt_test.xlsx')
fig_gantt = px.timeline(
                        df1, x_start='开始时间',x_end='结束时间',y='病例',color='地点',facet_col='工作岗位',
                        hover_name='病例',title='6031xx时空轨迹')

fig_gantt.update_xaxes(rangeslider_visible=True) # 添加x轴滑动条

fig_gantt.write_html('gantt.html')
fig_gantt

In [7]:
fig_gantt_2 = px.timeline(
                        df1, x_start='开始时间',x_end='结束时间',y='病例',color='地点', # facet_col='工作岗位',
                        hover_name='病例',title='5031xx时空轨迹')

fig_gantt_2.update_xaxes(rangeslider_visible=True) 

fig_gantt_2.write_html('gantt_2.html')
fig_gantt_2

### 2.详细查看一下它提供的内置的色谱、色序等详细信息

In [3]:
# 内置的色谱、色序等
px.colors.qualitative.swatches()
# px.colors.sequential.swatches()
# px.colors.diverging.swatches()
# px.colors.cyclical.swatches()
# px.colors.colorbrewer.swatches()
# px.colors.cmocean.swatches()
# px.colors.carto.swatches()

### 3.接下来，基于内置的人口寿命数据集gapminder来进行实践讲解

In [4]:
# 概览
print(px.data.gapminder.__doc__)
px.data.gapminder().head()


    Each row represents a country on a given year.

    https://www.gapminder.org/data/

    Returns:
        A `pandas.DataFrame` with 1704 rows and the following columns:
        `['country', 'continent', 'year', 'lifeExp', 'pop', 'gdpPercap',
        'iso_alpha', 'iso_num']`.
        If `datetimes` is True, the 'year' column will be a datetime column
        If `centroids` is True, two new columns are added: ['centroid_lat', 'centroid_lon']
        If `year` is an integer, the dataset will be filtered for that year
    


,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4


In [2]:
# 显示2007年按国家/地区的人均预期寿命和人均GDP之间的趋势
gapminder = px.data.gapminder()
gapminder2007 = gapminder.query('year == 2007')
px.scatter(gapminder2007,x='gdpPercap',y='lifeExp')

In [6]:
# 通过大陆颜色来区分
px.scatter(gapminder2007,x='gdpPercap',y='lifeExp',color='continent')

In [7]:
# 通过人口总数来区分
px.scatter(gapminder2007,x='gdpPercap',y='lifeExp',color='continent',size='pop',size_max=100,hover_name='country')

In [6]:
# 划分各大洲
px.scatter(gapminder2007,x='gdpPercap',y='lifeExp',color='continent',size='pop',size_max=100,hover_name='country',
            facet_col='continent',log_x=True) # facet_row='iso_alpha' 分面行

In [20]:
# 观看时间推移的变化趋势
px.scatter(gapminder,x='gdpPercap',y='lifeExp',color='continent',size='pop',size_max=100,hover_name='country',
            animation_frame='year',animation_group ='country',
            range_x=[0,70000],range_y=[25,100],
            labels=dict(pop="Population", gdpPercap="GDP per Capita", lifeExp="Life Expectancy"))

In [16]:
px.scatter(gapminder,x='gdpPercap',y='lifeExp',color='continent',size='pop',size_max=100,hover_name='country',
            animation_frame='year',animation_group ='country',facet_col='continent',
            range_y=[25,100],log_x=True,
            labels=dict(pop="Population", gdpPercap="GDP per Capita", lifeExp="Life Expectancy"))

In [22]:
# 线图
px.line(gapminder,x='year',y='lifeExp',color='continent',line_group='country',hover_name='country',
        line_shape='spline',render_mode='svg')

In [23]:
# 借助空间地图来更加直观展示
px.scatter_geo(gapminder2007,locations='iso_alpha',color='continent',hover_name='country',size='pop',
                animation_frame='year',projection='natural earth')

In [24]:
px.line_geo(gapminder2007, locations='iso_alpha',color='continent',projection='orthographic')

In [17]:
fig_animation = px.choropleth(gapminder,locations='iso_alpha',color='lifeExp',hover_name='country',animation_frame='year')
fig_animation.write_html('image/www.html')
fig_animation

In [25]:
# 旭日图
fig_xuri = px.sunburst(gapminder2007,path=['continent','country'],values='pop',color='lifeExp',hover_data=['iso_alpha'])
fig_xuri.write_image('image/xuri.svg',width=2160,height=1024) # 可选参数
fig_xuri.write_image('image/xuri.pdf')
fig_xuri

In [8]:
px.treemap(gapminder2007, path=[px.Constant('world'), 'continent', 'country'], values='pop',color='lifeExp', hover_data=['iso_alpha'])

In [12]:
px.icicle(gapminder2007, path=[px.Constant('world'), 'continent', 'country'], values='pop',color='lifeExp', hover_data=['iso_alpha'])

In [10]:
# 漏斗图
data_funnel = dict(number=[1000,800,400,200,100,30],stage=['UV','搜索','搜藏','加购','下单','付款'])
px.funnel(data_funnel,x='number',y='stage',hover_name='stage') # color='number'

### 4.使用内置iris数据

In [12]:
# 查看数据文档
print(px.data.iris.__doc__)
iris = px.data.iris()
iris


    Each row represents a flower.

    https://en.wikipedia.org/wiki/Iris_flower_data_set

    Returns:
        A `pandas.DataFrame` with 150 rows and the following columns:
        `['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species', 'species_id']`.


,sepal_length,sepal_width,petal_length,petal_width,species,species_id
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1
2,4.7,3.2,1.3,0.2,setosa,1
3,4.6,3.1,1.5,0.2,setosa,1
4,5.0,3.6,1.4,0.2,setosa,1
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,3
146,6.3,2.5,5.0,1.9,virginica,3
147,6.5,3.0,5.2,2.0,virginica,3
148,6.2,3.4,5.4,2.3,virginica,3


In [13]:
px.scatter(iris,  # 数据集
                x="sepal_width", # 横坐标
                y="sepal_length",  # 纵坐标
                color="species",  # 颜色
                marginal_x="box",  # 横坐标直方图
                marginal_y="violin",   # 细条图
                trendline="ols"  # 趋势线
                )

In [14]:
px.scatter_matrix(iris,  # 数据
                    dimensions=["sepal_width","sepal_length","petal_width","petal_length"],  # 维度选择
                    color="species")  # 颜色

In [15]:
px.parallel_coordinates(iris,   # 数据集
                        color="species_id",  # 颜色
                        labels={"species_id":"Species",  # 各种标签值
                                "sepal_width":"Sepal Width",
                                "sepal_length":"Sepal Length",
                                "petal_length":"Petal Length",
                                "petal_width":"Petal Width"},
                        color_continuous_scale=px.colors.diverging.Tealrose,
                        color_continuous_midpoint=2)

In [16]:
px.density_contour(iris,  # 绘图数据集
                    x="sepal_width",  # 横坐标
                    y="sepal_length",  # 纵坐标值
                    color="species"  # 颜色
                    )

### 5.使用小费数据集


In [17]:
tips = px.data.tips()
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [21]:
# 并行类别图
px.parallel_categories(tips,  # 数据集 
                            color="size",  # 颜色
                            color_continuous_scale=px.colors.sequential.Inferno)  # 颜色变化取值

In [22]:
# 柱状图

px.bar(tips,x='sex',y='total_bill',color='smoker',barmode='group')

In [23]:
px.bar(tips,  # 数据集
        x="sex",  # 横轴
        y="total_bill",  # 纵轴
        color="smoker",  # 颜色参数取值
        barmode="group",  # 柱状图模式取值
        facet_row="time",  # 行取值
        facet_col="day",  # 列元素取值
        category_orders={"day": ["Thur","Fri","Sat","Sun"],  # 分类顺序
                            "time":["Lunch", "Dinner"]}
        )

### 6.风向气象数据集

In [24]:
# 极坐标图
wind = px.data.wind()
wind.head()

,direction,strength,frequency
0,N,0-1,0.5
1,NNE,0-1,0.6
2,NE,0-1,0.5
3,ENE,0-1,0.4
4,E,0-1,0.4


In [27]:
# 线性极坐标图
px.line_polar(wind, r='frequency', # 半径值
                theta="direction",  # 角度
                color="strength",  # 颜色
                line_close=True,  # 线性闭合
                color_discrete_sequence=px.colors.sequential.Plasma_r)  # 颜色变化

In [29]:
# 柱状极坐标
px.bar_polar(wind,   # 数据集
                r="frequency",   # 半径
                theta="direction",  # 角度
                color="strength",  # 颜色
                template="plotly_dark",  # px中存在3种主题：plotly,plotly_whiter,plotly_dark
                color_discrete_sequence=px.colors.sequential.Plasma_r)  # 颜色变化